### Comparing GARCH conditional volatility to empirical rolling window vol

In [ ]:
from arch import arch_model
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("utils")
import compare_strategies as cs
import dynamic_backtesting as db
from matplotlib.dates import DateFormatter
from matplotlib.dates import YearLocator
from cycler import cycler
import numpy as np
plt.style.use("seaborn")
plt.rcParams.update({
   "text.usetex": True,
   "font.family": ['Latin Modern Roman'],
   "xtick.labelsize": 16.0,
   "ytick.labelsize": 16.0,
   "axes.labelsize":  16.0,
   "legend.fontsize": 16.0
}) 
import yfinance
import seaborn as sns

In [ ]:
returns = yfinance.download("IVV", start="2008-01-01", end="2021-10-02")['Adj Close']

In [ ]:
pct_return = returns.pct_change().iloc[1:]*100

In [ ]:
am = arch_model(pct_return)
res_vanilla = am.fit(disp="off")

In [ ]:
res_vol=pd.DataFrame(res_vanilla.conditional_volatility)
res_vol["Sample Volatility"]=pct_return.std()
res_vol=res_vol.iloc[:-1]

In [ ]:
res_vol.iloc[:2462,:]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 7), dpi=300)
ax.plot(res_vol.iloc[:2462,:])
ax.set_ylabel(r"\textbf{Standard deviation, $\hat{\sigma}$", loc='top', rotation = 0, labelpad = -165,size=14) 
ax.legend(fontsize=12)
ax.set_xlim("2008-01-02","2017-10-11")
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
plt.legend([r"\textbf{GARCH(1,1) cond. volatility}",r"\textbf{Sample volatility estimate}"])
fig.savefig("../Description of data/GarchvsEmpirical_vol.png",bbox_inches = 'tight')

### Using it for backtesting

In [ ]:
# Get return data
tickers = ['IVV', 'TLT', 'BZ=F', 'GC=F']
start="2008-01-01"
end="2021-10-02"
number_of_out_of_sample_days=250*4
len_rolling_window = 250
p = len(tickers)

out_of_sample, in_sample, sigmas, residuals, params_dict = db.split_fit_parse(tickers, start, end, number_of_out_of_sample_days, model_type="sGARCH11")

### First, no regularization (dont use this)

In [ ]:
# GARCH cond var estimate
Omega_ts_no_reg = db.calc_Omega_ts(out_of_sample_returns=out_of_sample, in_sample_returns=in_sample,
                             in_sample_sigmas=sigmas, in_sample_residuals=residuals, **params_dict, regularizer=0.0)

In [ ]:
GARCH_weights_no_reg = db.calc_weights_garch_no_trading_cost(Omega_ts_no_reg)

weight_index = pd.to_datetime(in_sample.index[[-1]].union(out_of_sample.index))
GARCH_weights_no_reg = pd.DataFrame(GARCH_weights_no_reg, columns=tickers, index=weight_index)

In [ ]:
# Construct dataset with burn-in period 
returns_for_rolling = pd.concat([in_sample.iloc[-len_rolling_window-1:-1], out_of_sample])

In [ ]:
# Naive rolling window estimate
rolling_estimate = returns_for_rolling.rolling(len_rolling_window).cov().values[(len_rolling_window-1)*p:]
naive_covars = []

for i, val in enumerate(rolling_estimate):
    if i % p == 0 and i !=0:
        naive_covars.append(rolling_estimate[i-p:i])

In [ ]:
rolling_weights_no_reg = pd.DataFrame(db.calc_weights_garch_no_trading_cost(naive_covars, False),
                            index=GARCH_weights_no_reg.index,
                            columns=GARCH_weights_no_reg.columns)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.set_prop_cycle(cycler(color=['red', 'blue', 'green']))
ax.plot(rolling_weights_no_reg, linestyle="--", alpha=0.8)
ax.plot(GARCH_weights_no_reg, alpha=0.8)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))

In [ ]:
cum_returns_GARCH_no_reg, perf_GARCH_no_reg = cs.performance_table(GARCH_weights_no_reg, out_of_sample,
                                                                   Omega_ts_no_reg, strategy_name="GARCH vol, not regularized")

In [ ]:
cum_returns_rolling_no_reg, perf_rolling_no_reg = cs.performance_table(rolling_weights_no_reg, out_of_sample,
                                                                       Omega_ts_no_reg, strategy_name="Rolling vol, not regularized")

Result: Rolling window estimate actually just as good as GARCH before trading costs - and clearly beating it after trading costs

### Now, we do the same but with a 50% regularization applied

##### Regularized static estimate

In [ ]:
# Regularize vol estimate by 50%
regularizer = 0.5

static_covar = in_sample.cov()
reg_target = np.zeros(static_covar.shape)+np.diag(np.diag(static_covar))
static_covar_reg = regularizer*reg_target + (1-regularizer)*static_covar

# Repeat this estimate 
static_covar_reg = np.array([static_covar_reg for static_covar in naive_covars])

In [ ]:
static_covar_weights_reg = pd.DataFrame(db.calc_weights_garch_no_trading_cost(static_covar_reg, False),
                            index=GARCH_weights_no_reg.index,
                            columns=GARCH_weights_no_reg.columns)

##### Regularizing rolling estimate

In [ ]:
naive_covars_reg = [(np.zeros((naive_covars[0].shape))+np.diag(np.diag(covar_estimate))) * regularizer + covar_estimate * (1 - regularizer) for covar_estimate in naive_covars]



In [ ]:
rolling_weights_reg = pd.DataFrame(db.calc_weights_garch_no_trading_cost(naive_covars_reg, False),
                            index=GARCH_weights_no_reg.index,
                            columns=GARCH_weights_no_reg.columns)

##### Regularizing GARCH

In [ ]:
# GARCH cond var estimate
Omega_ts_reg = db.calc_Omega_ts(out_of_sample_returns=out_of_sample, in_sample_returns=in_sample,
                             in_sample_sigmas=sigmas, in_sample_residuals=residuals, **params_dict, regularizer=0.5)
GARCH_weights_reg = db.calc_weights_garch_no_trading_cost(Omega_ts_reg)

weight_index = pd.to_datetime(in_sample.index[[-1]].union(out_of_sample.index))
GARCH_weights_reg = pd.DataFrame(GARCH_weights_reg, columns=tickers, index=weight_index)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.set_prop_cycle(cycler(color=['red', 'blue', 'green','black']))
ax.plot(rolling_weights_reg, linestyle="--", alpha=0.8)
ax.plot(GARCH_weights_reg, alpha=0.8)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r"\textbf{Weight, $v_t$}", loc='top', rotation = 0, labelpad = -108,size=14) 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.set_prop_cycle(cycler(color=['red', 'blue', 'green','black']))
ax.plot(static_covar_weights_reg, linestyle="--", linewidth=2.5)
ax.plot(GARCH_weights_reg, alpha=0.8,linewidth=0.8)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_xlim("2017-10-08","2021-10-02")
ax.set_ylim(-0.05,0.8)
plt.legend(["_Hidden Label","_Hidden Label","_Hidden Label","_Hidden Label",r"\textbf{S\&P 500}",r"\textbf{20+ Year Treasury Bonds}",r"\textbf{Brent Crude Oil Futures}",r"\textbf{Gold Futures}"],loc="upper left",ncol=2,fontsize=14)
ax.set_ylabel(r"\textbf{Weight, $v_t$}", loc='top', rotation = 0, labelpad = -95,size=14)
fig.savefig("../Description of data/GarchVsempirical_weight.png",bbox_inches = 'tight')

In [ ]:
cum_returns_static_reg, perf_static_reg = cs.performance_table(static_covar_weights_reg, out_of_sample, Omega_ts_reg, strategy_name="Static vol, regularized")

In [ ]:
cum_returns_rolling_reg, perf_rolling_reg = cs.performance_table(rolling_weights_reg, out_of_sample, Omega_ts_reg, strategy_name="Rolling vol, regularized")

In [ ]:
cum_returns_GARCH_reg, perf_GARCH_reg = cs.performance_table(GARCH_weights_reg, out_of_sample, Omega_ts_reg, strategy_name="GARCH vol, regularized")

In [ ]:
perf_rolling_reg

In [ ]:
print(perf_GARCH_reg.to_latex())

In [ ]:
print(perf_static_reg.to_latex())